In [1]:
# Start by importing numpy
import numpy as np

In [2]:
# Evaluate the starting population
def calc_fitness(pop, item_values):
    fitness = pop @ item_values
    return fitness

# Choose the parents
n_mating = 4

def select_parents(pop, pop_fitness):
    idx = np.argsort(pop_fitness)
    idx = idx[::-1]
    parents = pop[idx]
    
    # Select best of pop
    parents = parents[:n_mating]
    return parents

In [3]:
# Recombination
n_offspring = 3
crossover_idx = 3

def crossover(parents):
    offspring = np.empty((n_offspring, n_items))
    
    for k in range(n_offspring):        
        # Index of the first parent to mate.
        p1_idx = k%n_mating

        # Index of the second parent to mate.
        p2_idx = (k+1)%n_mating
        
        offspring[k, :crossover_idx] = parents[p1_idx, :crossover_idx]
        offspring[k, crossover_idx:] = parents[p2_idx, crossover_idx:]
        
    return offspring

In [4]:
# Mutation
mutation_prob = 0.5
def mutation(offspring):
    random_mutator = np.random.uniform(0.0, 1.0, (n_offspring,))
    
    idx = random_mutator > mutation_prob
    number_of_nonzeros = np.count_nonzero(idx)
    mutated_offspring = offspring.copy()
    mutated_offspring[idx] += np.round(np.random.uniform(-1,1,(number_of_nonzeros,n_items)))
    idx = mutated_offspring < 0
    mutated_offspring[idx] = 0
    return mutated_offspring

In [5]:
def calc_weight(pop, item_weights):
    weights = pop @ item_weights
    return weights

In [6]:
# eliminate members that don't meet weight constraint
def weight_constraint(pop, capacity, item_weights):
    idx = (calc_weight(pop, item_weights) <= capacity)
    return pop[idx]

In [7]:
# rank members of big population and cut down to pop_size
def environmental_selection(pop, pop_fitness, pop_size):
    idx = np.argsort(pop_fitness)
    idx = idx[::-1]
    pop = pop[idx]
    
    # Select best of pop
    pop = pop[0:pop_size]

    return pop

In [8]:
def generate_initial_pop(data_set, pop_size):
    
    # Load the data for the problem
    my_data = np.load(data_set)
    bag_capacity = int(my_data['capacity'])
    n_items = int(my_data['n_items'])
    item_values = my_data['item_values']
    item_weights = my_data['item_weights']
    
    dofs_in_pop = (pop_size,n_items)

    # starting population; each row is a member, each col an item
    mu = 0.0 # mean
    sigma = 0.3 # st. dev, spread
    pop = np.absolute(np.round(np.random.normal(loc = mu, scale = sigma, size = dofs_in_pop))) 
    pop = weight_constraint(pop, bag_capacity, item_weights)
    
    # make sure all the starting pop meets weight constraint
    while pop[:,1].size < pop_size:
        dofs_in_pop = (pop_size-pop[:,1].size,n_items)
        new_members = np.absolute(np.round(np.random.normal(loc = mu, scale = sigma, size = dofs_in_pop)))
        pop = np.vstack((pop,new_members))
        pop = weight_constraint(pop, bag_capacity, item_weights)
    
    return pop

In [9]:
# dataset A: algorithm to find optimal solution
my_data = np.load('A.npz')
bag_capacity = int(my_data['capacity'])
n_items = int(my_data['n_items'])
item_values = my_data['item_values']
item_weights = my_data['item_weights']

pop_size = 10
pop = generate_initial_pop('A.npz', pop_size)

# iterate to approach optimal solution
n_generations = 10000
    
for i in range(n_generations):
    parents = select_parents(pop,calc_fitness(pop, item_values))
    offspring = crossover(parents)
    mutated_offspring = mutation(offspring)
    pop = np.vstack((pop, mutated_offspring))
    pop = weight_constraint(pop, bag_capacity, item_weights)
    pop_A = environmental_selection(pop, calc_fitness(pop, item_values), pop_size)

In [10]:
print(pop_A)
print(calc_fitness(pop_A, item_values))
print(calc_weight(pop_A, item_weights))

[[0. 2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  1. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.

In [11]:
# dataset B: algorithm to find optimal solution
my_data = np.load('B.npz')
bag_capacity = int(my_data['capacity'])
n_items = int(my_data['n_items'])
item_values = my_data['item_values']
item_weights = my_data['item_weights']

pop_size = 10
pop = generate_initial_pop('B.npz', pop_size)

# iterate to approach optimal solution
n_generations = 10000
    
for i in range(n_generations):
    parents = select_parents(pop,calc_fitness(pop, item_values))
    offspring = crossover(parents)
    mutated_offspring = mutation(offspring)
    pop = np.vstack((pop, mutated_offspring))
    pop = weight_constraint(pop, bag_capacity, item_weights)
    pop_B = environmental_selection(pop, calc_fitness(pop, item_values), pop_size)

In [12]:
print(pop_B)
print(calc_fitness(pop_B, item_values))
print(calc_weight(pop_B, item_weights))
print(bag_capacity)

[[0. 0. 9. 0. 1. 2. 0. 0. 1. 0. 2. 0. 6. 0. 0. 2. 6. 1. 0. 0. 0. 2. 2. 0.
  0. 0. 0. 0. 0. 0. 0. 8. 1. 7. 0. 0. 0. 1. 0. 3. 0. 0. 1. 1. 0. 0. 0. 1.
  1. 1.]
 [0. 0. 9. 0. 0. 4. 0. 0. 0. 0. 3. 0. 8. 0. 0. 1. 5. 1. 0. 0. 0. 1. 2. 0.
  1. 1. 0. 0. 0. 1. 0. 6. 2. 6. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 2.]
 [0. 0. 9. 0. 0. 4. 0. 0. 0. 0. 3. 0. 8. 0. 0. 1. 5. 1. 0. 0. 0. 1. 2. 0.
  1. 1. 0. 0. 0. 1. 0. 6. 2. 6. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 2.]
 [0. 0. 9. 0. 0. 4. 0. 0. 0. 0. 3. 0. 8. 0. 0. 1. 5. 1. 0. 0. 0. 1. 2. 0.
  1. 1. 0. 0. 0. 1. 0. 6. 2. 6. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 2.]
 [0. 0. 9. 0. 0. 4. 0. 0. 0. 0. 3. 0. 8. 0. 0. 1. 5. 1. 0. 0. 0. 1. 2. 0.
  1. 1. 0. 0. 0. 1. 0. 6. 2. 6. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 2.]
 [0. 0. 9. 0. 0. 4. 0. 0. 0. 0. 3. 0. 8. 0. 0. 1. 5. 1. 0. 0. 0. 1. 2. 0.
  1. 1. 0. 0. 0. 1. 0. 6. 2. 6. 0. 0. 0. 1. 0. 3. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 2.]
 [0. 0. 9. 0. 0. 4. 0. 0. 0. 0. 3. 0. 8. 0. 0. 1. 5. 1. 0.